# Computing the bicrystallographic aspects of Misorientations (FCC lattice)

In [15]:
from sympy.matrices import Matrix, eye, zeros
import numpy as np

import byxtal.lattice as gbl
import byxtal.csl_utility_functions as cuf
import byxtal.find_csl_dsc as fcd

import byxtal.tools as bxt
import byxtal.misorient_fz as mfz
import byxtal.disorient_symm_props as dsp


Define the lattice: Here we use the `lattice` module and use the default option. The default option will initialize a cubic lattice with $a = 1$.

In [16]:
l1 = gbl.Lattice()
sig_type = 'common'
# l_p_po = Matrix(l1.l_p_po)
l_p_po = l1.l_p_po

First, initialize the $\Sigma$ number of interest. Note that for cubic crystals, CSL rotations exist only for odd-values of $\Sigma$.

In [17]:
sig_num = 651

The function `csl_rotations` in the module `byxtal.csl_utility_functions`, will compute the $\Sigma$ rotation (a misorientation) in the reference frame of the primitive cell.

In [18]:
s1 = cuf.csl_rotations(sig_num, sig_type, l1);
# print(s1)

As can be obsrved from the output above, there exist multiple rotations that correspond to a given $\Sigma$ number. This is usually the case when $\Sigma$ is large. In the following, we will compute the bycrystallography properties for one of the misorientations (e.g. corresponding to the second rotation in the array). The bycrystallography properties of interest are as follows:
+ The Coincidence Site Lattice (CSL). For more details, refer to this [tutorial](https://www.tf.uni-kiel.de/matwis/amat/def_en/kap_7/backbone/r7_1_2.html).
+ The DSC lattice that defines the **D**isplacements that are **S**ymmetry **C**onserving (also referred to as the **D**isplacement **S**hift **C**omplete). For more details, refer to this [tutorial](https://www.tf.uni-kiel.de/matwis/amat/def_en/kap_7/backbone/r7_1_3.html)
+ The boundary-plane symmetry group, and the symmetry axes. For more details refer to the following article:
    - [**Symmetries in the representation of grain boundary-plane distributions**](https://www.tandfonline.com/doi/full/10.1080/14786435.2012.722700?casa_token=4nDY3LLK9vQAAAAA%3AgECePf6JCdOPx6K_7cfxrJATdUAwMOSe3-F5NIdpjuI26uAPQfkkSTa61np1AWoDGQ3wcJrh3SS9). Patala, S., & Schuh, C. A. (2013). . Philosophical Magazine, 93(5), 524-573.

In [19]:
ct1 = 7
sig_id = str(sig_num)+'_'+str(ct1)
print(sig_id)

651_7


The $\Sigma$-misorientation is a rotation operation that is defined in the $P$ lattice reference frame and is computed as shown in the code snippet below. We use the variable `T_p1top2_p1` denote the **T**ransformation (`T`) from the $P1$ lattice to the $P2$ lattice (`p1top2`) expressed in the $P1$ reference frame (`p1`).

In [20]:
T_p1top2_p1 = s1['N'][ct1-1]/s1['D'][ct1-1]
print(T_p1top2_p1)

[[ 0.7281106  -0.46082949  0.29339478]
 [ 0.65437788  1.0921659   0.14132104]
 [-0.48387097  0.12903226  0.7311828 ]]


The **CSL** and the **DSC** lattice are determined using the `find_csl_dsc` function in the `byxtal.find_csl_dsc` module. The lattices are expressed using the basis vectors of the primitive cell $(P1)$. The following varibales are used:
+ `l_csl_p`: The lattice (`l`) of the CSL (`csl`) expressed in the primitive reference frame (`p`).
+ `l_dsc_p`: The lattice (`l`) of the DSC (`dsc`) expressed in the primitive reference frame (`p`).

In [21]:
# l_csl_p, l_dsc_p = fcd.find_csl_dsc(l_p_po, T_p1top2_p1, 1e-6)
# l_p_po = Matrix(l_p_po)
T_p1top2_p1 = np.array(T_p1top2_p1, dtype='double')
TI_p1top2_p1 = T_p1top2_p1*sig_num
TI_p1top2_p1 = Matrix((np.around(TI_p1top2_p1)).astype(int))
# l_csl_p = fcd.csl_finder(TI_p1top2_p1, sig_num, l_p_po, 1e-6)
l_csl_p = fcd.csl_finder(T_p1top2_p1, l_p_po, 1e-6)
print(l_csl_p)
# csl_mats[sig_id] = l_csl_p; 

[[  1.   1. -12.]
 [  2.  -7.   2.]
 [ -9.   3.   1.]]


Now, we will determine the boundary-plane symmetry group (and the axes) for the $\Sigma$ misorientation. The following steps are followed:
+ First, we need to cnvert the rotation in the $P$ reference frame (`T_p1top2_p1`) to $PO$ reference frame (`T_p1top2_po1`).
+ The misorientation is converted to a quaternion using the `mat2quat` function in the `byxtal.tools` module.
+ The disorientation is obtained using the `misorient_fz` function in the `byxtal.misorient_fz` module.
+ The bicrystallography symmetry properties are computed using `disorient_symm_props` function in the `byxtal.disorient_symm_props` module. The outputs are stored in the following variables:
    - `bp_symm_grp`: The point group symmetry corresponding to the given misorientation.
    - `symm_grp_ax`: The axes of the symmetry point group are stored in a $3 \times 3$ matrix. The columns correspond to the $\hat{e}_x$, $\hat{e}_y$, $\hat{e}_z$ axes of the symmetry point group.

In [22]:
l_p_po = l1.l_p_po
l_po_p = np.linalg.inv(l_p_po)
T_p1top2_po1 = np.dot(l_p_po, np.dot(T_p1top2_p1, l_po_p))
quat1 = bxt.mat2quat(T_p1top2_po1)
dis_quat1 = mfz.misorient_fz(quat1, l1.cryst_ptgrp)
x_g, y_g, z_g, bp_symm_grp = dsp.disorient_symm_props(dis_quat1, l1.cryst_ptgrp)
symm_grp_ax = (np.vstack((x_g, y_g, z_g))).transpose()
print(bp_symm_grp)
print(symm_grp_ax)

Cs
[[ 0.1108548   0.02534562  0.3048507 ]
 [-0.3048507   0.00921659  0.1108548 ]
 [ 0.         -0.10522273  0.0831411 ]]


One could compute the bycrystallography properties of a large number of $\Sigma$ rotation by using the code snippets discussed above and putting all of them together in a for-loop. This is shown in the code below.

In [23]:
n1 = 1 
n2 = 10
sig_nums = 2*np.arange(n1,n2)+1

num_sigs = 0
sig_mats = {}
csl_mats = {}
# dsc_mats = {}
csl_bp_props = {}

for sig_num in sig_nums:
    s1 = cuf.csl_rotations(sig_num, sig_type, l1)
    for ct1 in range(np.shape(s1['N'])[0]):
        bp_symm_grp_props = {}
        sig_id = str(sig_num)+'_'+str(ct1+1)
        # print(sig_id)
        #### Store the sigma-misorientation (in 'p' reference frame)
        T_p1top2_p1 = s1['N'][ct1]/s1['D'][ct1]
        sig_mats[sig_id] = T_p1top2_p1

        # l_csl_p, l_dsc_p = fcd.find_csl_dsc(l_p_po, T_p1top2_p1, 1e-6)
        # l_p_po = Matrix(l_p_po)
        T_p1top2_p1 = np.array(T_p1top2_p1, dtype='double')
        TI_p1top2_p1 = T_p1top2_p1*sig_num
        TI_p1top2_p1 = Matrix((np.around(TI_p1top2_p1)).astype(int))
        # l_csl_p = fcd.csl_finder(TI_p1top2_p1, sig_num, Matrix(l_p_po), 1e-6)
        l_csl_p = fcd.csl_finder(T_p1top2_p1, l_p_po, 1e-6)
        csl_mats[sig_id] = l_csl_p 
        # dsc_mats[sig_id] = l_dsc_p

        #### Generate boundary-planpe orientations
        l_p_po = l1.l_p_po 
        l_po_p = np.linalg.inv(l_p_po)
        T_p1top2_po1 = np.dot(l_p_po, np.dot(T_p1top2_p1, l_po_p))

        ## Find the corresponding disorientation
        quat1 = bxt.mat2quat(T_p1top2_po1) 
        # print(quat1)
        dis_quat1 = mfz.misorient_fz(quat1, l1.cryst_ptgrp) 
        # print(dis_quat1)
        x_g, y_g, z_g, bp_symm_grp = dsp.disorient_symm_props(dis_quat1, l1.cryst_ptgrp)
        symm_grp_ax = (np.vstack((x_g, y_g, z_g))).transpose()
        bp_symm_grp_props['symm_grp_ax'] = symm_grp_ax
        bp_symm_grp_props['bp_symm_grp'] = bp_symm_grp
        csl_bp_props[sig_id] = bp_symm_grp_props

# import pickle as pkl
# pkl_name = 'cF_Id_csl_common_rotations.pkl'
# csl_props = {}
# csl_props['sig_mats'] = sig_mats
# csl_props['csl_mats'] = csl_mats
# # csl_props['dsc_mats'] = dsc_mats
# csl_props['csl_bp_props'] = csl_bp_props

# jar = open(pkl_name, 'wb')
# pkl.dump(csl_props, jar)
# jar.close()

In [24]:
sig_mats

{'3_1': array([[ 0.66666667, -0.33333333,  0.66666667],
        [ 0.66666667,  0.66666667, -0.33333333],
        [-0.33333333,  0.66666667,  0.66666667]]),
 '5_1': array([[ 0.8, -0.4,  0.2],
        [ 0.6,  1.2,  0.4],
        [-0.6, -0.2,  0.6]]),
 '7_1': array([[ 0.85714286, -0.28571429,  0.42857143],
        [ 0.42857143,  0.85714286, -0.28571429],
        [-0.28571429,  0.42857143,  0.85714286]]),
 '9_1': array([[ 0.55555556, -0.66666667,  0.        ],
        [ 0.66666667,  1.        ,  0.        ],
        [-0.11111111,  0.33333333,  1.        ]]),
 '11_1': array([[ 0.36363636, -0.81818182,  0.        ],
        [ 0.81818182,  0.90909091,  0.        ],
        [-0.09090909,  0.45454545,  1.        ]]),
 '13_1': array([[ 0.92307692, -0.23076923,  0.30769231],
        [ 0.30769231,  0.92307692, -0.23076923],
        [-0.23076923,  0.30769231,  0.92307692]]),
 '13_2': array([[ 0.92307692, -0.23076923,  0.15384615],
        [ 0.38461538,  1.15384615,  0.23076923],
        [-0.3846153